In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Run this
#!pip install albumentations > /dev/null
#!pip install -U efficientnet==0.0.4
!pip install keras_efficientnets
#%tensorflow_version 1.x
import numpy as np
import pandas as pd
import gc
import keras

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split,StratifiedKFold

from skimage.transform import resize
import tensorflow as tf
import keras.backend as K
from keras.losses import binary_crossentropy

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import  ModelCheckpoint
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization, Multiply, Subtract, AveragePooling2D
from tqdm import tqdm_notebook
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils, plot_model
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras import backend as K
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.losses import binary_crossentropy
import keras.callbacks as callbacks
from keras.callbacks import Callback
from keras.applications.xception import Xception
from keras.layers import multiply


from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects

from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.regularizers import l2
from keras.layers.core import Dense, Lambda
from keras.layers.merge import concatenate, add
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Permute
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator

import glob
import shutil
import os
import random


seed = 10
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

#tf.set_random_seed(seed)
    
%matplotlib inline

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [ ]:
ct0 = '/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/SEVERITY/CT 0'
ct1 = '/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/SEVERITY/CT 1'
ct2 = '/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/SEVERITY/CT 2'
ct3 = '/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/SEVERITY/CT 3'

In [ ]:
#only for cases other than 0 vs 1

dist_0 = 135
dist_1 = 135


ct0_num = len(os.listdir(ct0))
ct1_num = len(os.listdir(ct1))
n0 = np.linspace(0, ct0_num-1, ct0_num, dtype=int) 
n1 = np.linspace(0, ct1_num-1, ct1_num, dtype=int) 

random.shuffle(n0)
random.shuffle(n1)

n0 = n0[:dist_0]
n1 = n1[:dist_1]


X = np.empty((dist_0+dist_1, 256, 256, 3, 30) , dtype = np.float64)
Y = np.empty(dist_0+dist_1, dtype = int)

import cv2
for i in range(dist_0):
  #a0_1 = os.path.join (ct0 , os.listdir(ct1)[i - bunch/4]) #patient
  a0_1 = os.path.join (ct0 , os.listdir(ct0)[n0[i]])
  print(a0_1)
  a0_1_1 = np.sort(random.sample(os.listdir(a0_1), 30)) # slice
  Y[i] = 0
  print(Y[i])
  for j in range (len(a0_1_1)):
    a011_img = os.path.join(a0_1, a0_1_1[j])
    a011_img = cv2.imread(a011_img)
    a011_img = resize(a011_img, (256, 256,3), anti_aliasing=True)
    X[i, :,:,: ,j] = a011_img
    print("patient : " + str(i) + " slice : " +str(j))
  
k= i+1

# CELL 2
for i in range(dist_1):
  #a0_1 = os.path.join (ct1 , os.listdir(ct1)[i - bunch/4]) #patient
  a0_1 = os.path.join (ct1 , os.listdir(ct1)[n1[i]])
  print(a0_1)
  a0_1_1 = np.sort(random.sample(os.listdir(a0_1), 30)) # slice
  Y[k+i] = 1
  print(Y[k+i])
  for j in range (len(a0_1_1)):
    a011_img = os.path.join(a0_1, a0_1_1[j])
    a011_img = cv2.imread(a011_img)
    a011_img = resize(a011_img, (256, 256,3), anti_aliasing=True)
    X[k+i, :,:,: ,j] = a011_img
    print("patient : " + str(i) + " slice : " +str(j))


Streaming output truncated to the last 5000 lines.
patient : 113 slice : 23
patient : 113 slice : 24
patient : 113 slice : 25
patient : 113 slice : 26
patient : 113 slice : 27
patient : 113 slice : 28
patient : 113 slice : 29
/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/SEVERITY/CT 0/study_0024.nii.gz
0
patient : 114 slice : 0
patient : 114 slice : 1
patient : 114 slice : 2
patient : 114 slice : 3
patient : 114 slice : 4
patient : 114 slice : 5
patient : 114 slice : 6
patient : 114 slice : 7
patient : 114 slice : 8
patient : 114 slice : 9
patient : 114 slice : 10
patient : 114 slice : 11
patient : 114 slice : 12
patient : 114 slice : 13
patient : 114 slice : 14
patient : 114 slice : 15
patient : 114 slice : 16
patient : 114 slice : 17
patient : 114 slice : 18
patient : 114 slice : 19
patient : 114 slice : 20
patient : 114 slice : 21
patient : 114 slice : 22
patient : 114 slice : 23
patient : 114 slice : 24
patient : 114 slice : 25
patient : 114 slice : 26
patient : 114 s

In [ ]:
#only for the case 0 vs 1

test0 = [53,  17,  51,  59, 177, 134,  21, 178, 232,
        248, 180,  66, 200, 148, 132,  94, 150, 131,  86, 126,  99, 183,
        103, 156, 218, 171, 215]   
test1 = [273, 662,  49, 473, 251, 532, 152, 530, 444,
        679, 164, 298, 303, 306, 137, 633, 400, 453,   0, 606,  73, 255,
        614, 487, 556, 646, 210]

ct0_num = len(os.listdir(ct0))
ct1_num = len(os.listdir(ct1))
n0 = np.linspace(0, ct0_num-1, ct0_num, dtype=int) 
n1 = np.linspace(0, ct1_num-1, ct1_num, dtype=int) 

test0 = np.array(test0)
test1 = np.array(test1)

for i in range(len(test0)):
  n0 = n0[n0!=test0[i]]
  n1 = n1[n1!=test1[i]]

dist_0 = 135
dist_1 = 135

import random
random.shuffle(n0)
random.shuffle(n1)

n0 = n0[:dist_0]
n1 = n1[:dist_1]

dist_0-= len(test0)
dist_1-= len(test1)

X_train = np.empty((dist_0+dist_1, 256, 256, 3, 30) , dtype = np.float64)
Y_train = np.empty(dist_0+dist_1, dtype = int)

X_test = np.empty((len(test0)+len(test1), 256, 256, 3, 30) , dtype = np.float64)
Y_test = np.empty(len(test0)+len(test1), dtype = int)

import cv2
for i in range(dist_0):
  #a0_1 = os.path.join (ct0 , os.listdir(ct1)[i - bunch/4]) #patient
  a0_1 = os.path.join (ct0 , os.listdir(ct0)[n0[i]])
  print(a0_1)
  a0_1_1 = np.sort(random.sample(os.listdir(a0_1), 30)) # slice
  Y_train[i] = 0
  print("train")
  print(Y_train[i])
  for j in range (len(a0_1_1)):
    a011_img = os.path.join(a0_1, a0_1_1[j])
    a011_img = cv2.imread(a011_img)
    a011_img = resize(a011_img, (256, 256,3), anti_aliasing=True)
    X_train[i, :,:,: ,j] = a011_img
    print("patient : " + str(i) + " slice : " +str(j))
  
k= i+1

# CELL 2
for i in range(dist_1):
  #a0_1 = os.path.join (ct1 , os.listdir(ct1)[i - bunch/4]) #patient
  a0_1 = os.path.join (ct1 , os.listdir(ct1)[n1[i]])
  print(a0_1)
  a0_1_1 = np.sort(random.sample(os.listdir(a0_1), 30)) # slice
  Y_train[k+i] = 1
  print("train")
  print(Y_train[k+i])
  for j in range (len(a0_1_1)):
    a011_img = os.path.join(a0_1, a0_1_1[j])
    a011_img = cv2.imread(a011_img)
    a011_img = resize(a011_img, (256, 256,3), anti_aliasing=True)
    X_train[k+i, :,:,: ,j] = a011_img
    print("patient : " + str(i) + " slice : " +str(j))

for i in range(len(test0)):
  #a0_1 = os.path.join (ct0 , os.listdir(ct1)[i - bunch/4]) #patient
  a0_1 = os.path.join (ct0 , os.listdir(ct0)[test0[i]])
  print(a0_1)
  a0_1_1 = np.sort(random.sample(os.listdir(a0_1), 30)) # slice
  Y_test[i] = 0
  print("test")
  print(Y_test[i])
  for j in range (len(a0_1_1)):
    a011_img = os.path.join(a0_1, a0_1_1[j])
    a011_img = cv2.imread(a011_img)
    a011_img = resize(a011_img, (256, 256,3), anti_aliasing=True)
    X_test[i, :,:,: ,j] = a011_img
    print("patient : " + str(i) + " slice : " +str(j))
  
k= i+1

# CELL 2
for i in range(len(test1)):
  #a0_1 = os.path.join (ct1 , os.listdir(ct1)[i - bunch/4]) #patient
  a0_1 = os.path.join (ct1 , os.listdir(ct1)[test1[i]])
  print(a0_1)
  a0_1_1 = np.sort(random.sample(os.listdir(a0_1), 30)) # slice
  Y_test[k+i] = 1
  print("test")
  print(Y_test[k+i])
  for j in range (len(a0_1_1)):
    a011_img = os.path.join(a0_1, a0_1_1[j])
    a011_img = cv2.imread(a011_img)
    a011_img = resize(a011_img, (256, 256,3), anti_aliasing=True)
    X_test[k+i, :,:,: ,j] = a011_img
    print("patient : " + str(i) + " slice : " +str(j))


In [ ]:
np.unique(Y)

array([0, 1])

In [ ]:
# # CELL 3
# for i in range(200,250):
#   #a0_1 = os.path.join (ct1 , os.listdir(ct1)[i - bunch/4]) #patient
#   a0_1 = os.path.join (ct2 , os.listdir(ct2)[i-200])
#   print(a0_1)
#   a0_1_1 = np.sort(random.sample(os.listdir(a0_1), 30)) # slice
#   Y[i] = 1
#   print(Y[i])
#   for j in range (len(a0_1_1)):
#     a011_img = os.path.join(a0_1, a0_1_1[j])
#     a011_img = cv2.imread(a011_img)
#     a011_img = resize(a011_img, (256, 256,3), anti_aliasing=True)
#     X[i, :,:,: ,j] = a011_img
#     print("patient : " + str(i) + " slice : " +str(j))
    

In [ ]:
# # CELL 2
# for i in range(250,300):
#   #a0_1 = os.path.join (ct1 , os.listdir(ct1)[i - bunch/4]) #patient
#   a0_1 = os.path.join (ct3 , os.listdir(ct3)[i-250])
#   print(a0_1)
#   a0_1_1 = np.sort(random.sample(os.listdir(a0_1), 30)) # slice
#   Y[i] = 1
#   print(Y[i])
#   for j in range (len(a0_1_1)):
#     a011_img = os.path.join(a0_1, a0_1_1[j])
#     a011_img = cv2.imread(a011_img)
#     a011_img = resize(a011_img, (256, 256,3), anti_aliasing=True)
#     X[i, :,:,: ,j] = a011_img
#     print("patient : " + str(i) + " slice : " +str(j))
    

In [ ]:
def jahin_activation(x):
    return (1. / (1. + K.exp(5*(x-0.5))))

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


In [ ]:
def mse_block(x_input):  # this block takes a block of input shape (256, 256, nb_channels) )(256,256, 64)
  x_in1 = GlobalAveragePooling2D()(x_input)
  x_in1 = Dense(int(x_input.shape[-1]//4) , activation = 'relu')(x_in1)
  x_in1 = Dense(int(x_input.shape[-1]), activation = 'sigmoid')(x_in1)
  s1 = Multiply()([x_in1 , x_input])

  x_in2 = Conv2D(filters = int(x_input.shape[-1]), kernel_size = (3,3), strides = (2,2), padding = 'same')(x_input)
  x_in2 = Conv2DTranspose (filters = 1, kernel_size = (3,3), strides = (2,2), padding = 'same', activation = 'sigmoid')(x_in2)
  s2 = Multiply()([x_in2 , x_input])
  

  x_in3 = GlobalAveragePooling2D()(x_input)
  x_in3 = Dense(int(x_input.shape[-1]*4) , activation = 'relu')(x_in3)
  x_in3 = Dense(int(x_input.shape[-1]), activation = 'sigmoid')(x_in3)
  s3 = Multiply()([x_in3 , x_input])

  x_in4 = Conv2DTranspose (filters = int(x_input.shape[-1]), kernel_size = (3,3), strides = (2,2), padding = 'same')(x_input)
  x_in4 = Conv2D(filters = int(x_input.shape[-1]), kernel_size = (3,3), strides = (2,2), padding = 'same', activation = 'sigmoid')(x_in4)
  s4 = Multiply()([x_in4 , x_input])

  #s1_0 =  Dense(int(x_input.shape[-1]), activation = jahin_activation)(x_in1)
  #s1_0 =  Multiply()([s1_0 , x_input])
  #s1_0 =  Conv2D(filters = int(x_input.shape[-1]), kernel_size = (1,1), strides = (1,1), padding = 'same', activation= 'relu')(s1_0)
    
  s2_0 =  Conv2D(filters = int(x_input.shape[-1]), kernel_size = (3,3), strides = (1,1), padding = 'same', activation= jahin_activation)(x_in2)
  s2_0 =  Multiply()([s2_0 , x_input])
  s2_0 =  Conv2D(filters = int(x_input.shape[-1]), kernel_size = (1,1), strides = (1,1), padding = 'same', activation= 'relu')(s2_0)
  
  #s12_0 = Concatenate() ([s1_0, s2_0])
  #s12_0 = Conv2D (filters = int(x_input.shape[-1]), kernel_size = (1,1),strides = (1,1), padding = 'same' , activation = 'relu')(s12_0)
  out = Concatenate() ([s1, s3, s2, s4])
  out = Conv2D (filters = int(x_input.shape[-1]), kernel_size = (1,1),strides = (1,1), padding = 'same' , activation = 'relu')(out)
  out = Add()([out , x_input])
  #out = Subtract()([out, s2_0])
  
  return out


In [ ]:
def PYRAMID(x_input): # number of channels should be power of 2
  
  track_1 = Conv2D(filters = int(x_input.shape[-1]//4), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(x_input)

  track_2 = AveragePooling2D(pool_size=(2, 2) , strides=2, padding = 'same')(x_input)
  track_2 = Conv2D(filters = int(x_input.shape[-1]//4), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(track_2)
  track_2 = UpSampling2D((2,2))(track_2)
  #print(track_2.shape)

  track_3 = AveragePooling2D(pool_size=(4, 4) , strides=4, padding = 'same')(x_input)
  track_3 = Conv2D(filters = int(x_input.shape[-1]//4), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(track_3)
  track_3 = UpSampling2D((4,4))(track_3)
  #print(track_3.shape)

  track_4 = AveragePooling2D(pool_size=(8, 8) , strides=8 , padding = 'same')(x_input)
  track_4 = Conv2D(filters = int(x_input.shape[-1]//4), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(track_4)
  track_4 = UpSampling2D((8,8))(track_4)
  #print(track_4.shape)

  x_out_1 = Concatenate(axis = -1)([track_1, track_2 , track_3, track_4, x_input])
  

  x_out_1 = Conv2D(filters = int(x_out_1.shape[-1]//2), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(x_out_1)
  x_out_1 = Add()([x_input , x_out_1])
  return x_out_1

In [ ]:
#def m_RPP(x_input): # number of channels should be power of 2     # SNEAKY STUFF  # SORRY JAHIN
def m_RPP(x_input):
  
  track_1 = Conv2D(filters = int(x_input.shape[-1]//4), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(x_input)

  track_2 = AveragePooling2D(pool_size=(2, 2) , strides=2, padding = 'same')(x_input)
  track_2 = Conv2D(filters = int(x_input.shape[-1]//4), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(track_2)
  track_2 = UpSampling2D((2,2))(track_2)
  #print(track_2.shape)

  track_3 = AveragePooling2D(pool_size=(4, 4) , strides=4, padding = 'same')(x_input)
  track_3 = Conv2D(filters = int(x_input.shape[-1]//4), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(track_3)
  track_3 = UpSampling2D((4,4))(track_3)
  #print(track_3.shape)

  track_4 = AveragePooling2D(pool_size=(8, 8) , strides=8 , padding = 'same')(x_input)
  track_4 = Conv2D(filters = int(x_input.shape[-1]//4), kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(track_4)
  track_4 = UpSampling2D((8,8))(track_4)
  #print(track_4.shape)

  x_out_1 = Concatenate(axis = -1)([track_1, track_2 , track_3, track_4, x_input])
  

  x_out_1 = Conv2D(filters = 32, kernel_size=(1, 1), strides=1, padding='same', kernel_regularizer=l2(1e-3))(x_out_1)
  #x_out_1 = Add()([x_input , x_out_1])
  return x_out_1

In [ ]:
from keras_efficientnets import EfficientNetB5

backbone = EfficientNetB5(weights='imagenet',
                          include_top=False,
                          input_shape=(256, 256, 3))

backbone = Model(backbone.input, backbone.layers[515].output)
backbone.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 128, 128, 48) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [ ]:
def M_unet(backbone, heavier=0):
  a1 = backbone.layers[41].output   #128x128x144       |
  a2 = backbone.layers[115].output  #64x64x240         |
  a3 = backbone.layers[189].output  #32x32x384         |    <--encoder
  a4 = backbone.layers[397].output  #16x16x1056        |
  a5 = backbone.layers[-1].output   #8x8x1824          |


  
  def just_conv(x, filter_, ker=3, up=0):
      if up:  x= UpSampling2D((2,2))(x)
      x = Conv2D(filter_, (ker,ker), activation='relu', padding='same')(x)
      return x
  
  def manual_conv_up(x, sc, ker=3):
      x = Conv2D(int(x.shape[-1]//sc), (ker,ker), activation='relu', padding='same')(x)
      x= UpSampling2D((sc,sc))(x)
      return x

  factor=1
  if heavier: factor=2

  a10 = just_conv(a1, 16*factor, up=1)
  a20 = just_conv(a2, 32*factor, up=1)
  a30 = just_conv(a3, 64*factor, up=1)
  a40 = just_conv(a4, 128*factor, up=1)
  a50 = just_conv(a5, 256*factor, up=1)


  a10 = mse_block(a10)
  a20 = mse_block((a20))
  a30 = mse_block((a30))
  a40 = mse_block((a40))
  a50 = mse_block((a50))

   
  
  b6 = just_conv(a5, 256*factor)                                    #           |            
  b6 = just_conv(b6, 256*factor, up=1)

  b5 = concatenate([b6,a50], axis=-1)                               #           |
  b5 = just_conv(b5, 256*factor)
  b5 = just_conv(b5, 256*factor)

  b4 = concatenate([just_conv(b5, 256*factor, up=1), a40], axis=-1) #           |
  b4 = just_conv(b4, 128*factor)
  b4 = just_conv(b4, 128*factor)

  b3 = concatenate([just_conv(b4, 128*factor, up=1), a30], axis=-1) #           |  <--decoder
  b3 = just_conv(b3, 64*factor)
  b3 = just_conv(b3, 64*factor)

  b2 = concatenate([just_conv(b3, 64*factor, up=1), a20], axis=-1)  #           |
  b2 = just_conv(b2, 32*factor)
  b2 = just_conv(b2, 32*factor)

  b1 = concatenate([just_conv(b2, 32*factor, up=1), a10], axis=-1)  #           |
  b1 = just_conv(b1, 16*factor)
  b1 = just_conv(b1, 16*factor)                                     #           |

  b10 = mse_block((b1))
  b20 = mse_block((b2))
  b30 = mse_block((b3))
  b40 = mse_block((b4))
  b50 = mse_block((b5))

  #left side
  b10 = just_conv(b10, filter_=16, ker=1)

  b20 = just_conv(b20, filter_=16, ker=1)
  b20 = UpSampling2D((2,2))(b20)

  b30 = just_conv(b30, filter_=16, ker=1)
  b30 = UpSampling2D((4,4))(b30)

  b40 = just_conv(b40, filter_=16, ker=1)
  b40 = UpSampling2D((8,8))(b40)

  b50 = just_conv(b50, filter_=16, ker=1)
  b50 = UpSampling2D((16,16))(b50)

  b_all = concatenate([b10, b20, b30, b40, b50], axis=-1)

  #right side
  a10 = just_conv(a10, filter_=16, ker=1)

  a20 = just_conv(a20, filter_=16, ker=1)
  a20 = UpSampling2D((2,2))(a20)

  a30 = just_conv(a30, filter_=16, ker=1)
  a30 = UpSampling2D((4,4))(a30)

  a40 = just_conv(a40, filter_=16, ker=1)
  a40 = UpSampling2D((8,8))(a40)

  a50 = just_conv(a50, filter_=16, ker=1)
  a50 = UpSampling2D((16,16))(a50)

  a_all = concatenate([a10, a20, a30, a40, a50], axis=-1)
  
  
  a_b = concatenate([a_all, b_all], axis=-1)
  
  out = Conv2D(1, (1, 1), activation='sigmoid')(a_b)
  
  #out = Conv2D(1, (1, 1), activation='sigmoid')(b1)

  model= Model(backbone.input, out)

  return model



m_unet= M_unet(backbone,0)
m_unet.compile(optimizer=Adam(lr=1.5e-3), loss='binary_crossentropy', metrics=[dice_coef])
m_unet.load_weights ('/content/drive/My Drive/Data/m_unet_jahin_custAct_rpp+_mse_80by20_10.h5')

In [ ]:
m_unet.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 128, 128, 48) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [ ]:
a = m_unet.layers[-2].output
print(a.shape)


for layer in m_unet.layers:
	layer.trainable = False


temp_model = Model (inputs = m_unet.input, outputs = a )
temp_model.summary()

(None, 256, 256, 160)
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 128, 128, 48) 0           batch_normalization_1[0][0]      
______________________________________________________________________

In [ ]:
# def m_attention_block(slice_x):
#         basket=[]
#         y = temp_model(slice_x)
#         for i in range(0,5):
#           #name = "up"+str(i)
#           #out = unet.get_layer(name).output
#           #temp_model = Model(inputs = unet.input, outputs = out)
#           x = mse_block(y[i])          
#           #x = Add()([x, x2])

#           x = Conv2D(64, (1,1), activation= "relu", padding="same")(x)
          

#           while x.shape[1]!= 256: x= UpSampling2D((2,2))(x)
#           x = Conv2D(3, (3,3), activation= "sigmoid", padding="same")(x)
#           x = Multiply()([slice_x, x])
          
#           #x = model_vgg(x)
#           #x = Conv2D(64, (1,1), activation= "relu", padding="same")(x)
#           basket.append(x)
#         x = Concatenate(axis=-1)(basket)

        
#         return x


# x0 = Input((256,256,3))
# y0 = m_attention_block(x0)

# m_attention = Model(x0,y0)

In [ ]:
# def m_feature_extractor_block(slice_x):
#         basket=[]
#         y = temp_model(slice_x)
#         for i in range(0,5):
#           #name = "up"+str(i)
#           #out = unet.get_layer(name).output
#           #temp_model = Model(inputs = unet.input, outputs = out)
#           x = PYRAMID(y[i])
#           x = mse_block(x)          
#           #x = Add()([x, x2])

#           x = Conv2D(16, (1,1), activation= "relu", padding="same")(x)
          

#           while x.shape[1]!= 256: x= UpSampling2D((2,2))(x)
#           #x = Conv2D(3, (3,3), activation= "sigmoid", padding="same")(x)
#           #x = Multiply()([slice_x, x])
          
#           #x = model_vgg(x)
#           #x = Conv2D(64, (1,1), activation= "relu", padding="same")(x)
#           basket.append(x)
        
#         x = Concatenate(axis=-1)(basket)
        
#         return x


# x1 = Input((256,256,3))
# y1 = m_feature_extractor_block(x1)

# m_feat_ex = Model(x1,y1)

In [ ]:
#Run this
#21st July new classifier
from keras.layers import Multiply


def approach_1_block(in_shape):
    x_in = Input(in_shape)
    slice_num= x_in.shape[-1]
    
    branch = []
    
    

    for i in range(slice_num):
        print(i)
        x_out = Lambda(lambda x: x[:,:,:,:,i])(x_in)  
        x_out_1_1= temp_model(x_out)
        

        ##modify
        x_out_1 = Conv2D(64, (3,3), activation= "relu", padding="same")(x_out_1_1)
        x_out_1 = MaxPooling2D((2,2))(x_out_1)
        x_out_1 = Conv2D(128, (3,3), activation= "relu", padding="same")(x_out_1)
        x_out_1 = Conv2D(128, (3,3), activation= "relu", padding="same")(x_out_1)
        x_out_1 = MaxPooling2D((2,2))(x_out_1)
        
        x_out_1 = Conv2D(256, (3,3), activation= "relu", padding="same")(x_out_1)
        x_out_1 = MaxPooling2D((2,2))(x_out_1)
          
        x_out_1 = Conv2D(16, (3,3), activation= "relu", padding="same")(x_out_1)
        x_out_1 = MaxPooling2D((2,2))(x_out_1)
        x_out_1 = PYRAMID(x_out_1)

        
        branch.append(x_out_1)
    
    x = Concatenate(axis=-1)(branch)
    print(x.shape)

    x = Conv2D(256, (1,1), activation= "relu", padding="same")(x)
    
    x = mse_block(x)
    x = GlobalAveragePooling2D()(x)


    # #x = Flatten()(x)
    x = Dense(128, activation='relu')(x)  
    #x = Dense(64, activation='relu')(x)
    #x = Dense(1, activation='sigmoid')(x)
    x = Dense(1, activation='sigmoid')(x) # WE JUST CHANGED THIS LAYER
    # print(x.shape)


    
    model = Model(x_in, x)

    return model


In [ ]:
#Run this
model_approach= approach_1_block((256, 256, 3, 30))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
(None, 16, 16, 480)


In [ ]:
#Run this
model_approach.summary()
#plot_model(model_approach)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3,  0                                            
__________________________________________________________________________________________________
lambda_41 (Lambda)              (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_42 (Lambda)              (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_43 (Lambda)              (None, 256, 256, 3)  0           input_3[0][0]                    
____________________________________________________________________________________________

In [ ]:
#Run this
# import cv2
# for j in range(int(bunch_1*init), int(bunch_1*(init+1)+rest1)):
#     a1_1= os.path.join(a1, os.listdir(a1)[j])  #patients
#     a1_1_1= np.sort(random.sample(os.listdir(a1_1), 30))  #30 sampling
#     for m in range(len(a1_1_1)):
#         a111_im = os.path.join(a1_1, a1_1_1[m])
#         a111_im = cv2.imread(a111_im)
#         a111_im = resize(a111_im, (256, 256,3), anti_aliasing=True)
#         X[j, :,:,: ,m] = a111_im
#         print("patient : " + str(j) + " slice : " +str(m))
#         Y[j] = 1

# for j in range(int(bunch_1*(init+1)+rest1)+int(bunch_0*init),int(bunch_1*(init+1)+rest1)+int(bunch_0*(init+1)+rest2)):
#     a2_1= os.path.join(a2, os.listdir(a2)[j - int(bunch_0)])  #patients
#     a2_1_1= np.sort(random.sample(os.listdir(a2_1), 30))  #30 sampling
#     for m in range(len(a2_1_1)):
#         a211_im = os.path.join(a2_1, a2_1_1[m])
#         a211_im = cv2.imread(a211_im)
#         a211_im = resize(a211_im, (256, 256, 3), anti_aliasing=True)
#         X[j, :,:,:,m ] = a211_im
#         print("patient : " + str(j) + " slice : " +str(m))
#         Y[j] = 0



def shuffle_in_unison_scary(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)
    return a, b

X,Y = shuffle_in_unison_scary(X,Y)

In [ ]:
Y[-50:]

array([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0])

In [ ]:
#To train classifier
from keras.callbacks import *

#model_approach.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_approach.compile(optimizer = Adam(lr=5e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])

weight_saver = ModelCheckpoint('/content/drive/My Drive/Data/ct_0v1_rpp_included_1.h5', monitor='val_accuracy', 
                                save_best_only=True, save_weights_only=True, mode= 'max')

annealer = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.0001,mode= 'min')

hist = model_approach.fit(x = X , 
                          y = Y, 
                          batch_size = 2,
                          epochs = 80,
                          callbacks = [weight_saver , annealer],
                          validation_split = 0.2,
                          shuffle = False,
                          verbose = 1,)



Train on 216 samples, validate on 54 samples
Epoch 1/80
216/216 [==============================] - 791s 4s/step - loss: 3177.7724 - accuracy: 0.5370 - val_loss: 1.6813 - val_accuracy: 0.7222
Epoch 2/80
216/216 [==============================] - 311s 1s/step - loss: 1.7964 - accuracy: 0.6204 - val_loss: 1.5599 - val_accuracy: 0.8333
Epoch 3/80
216/216 [==============================] - 312s 1s/step - loss: 2.8424 - accuracy: 0.6157 - val_loss: 1.4897 - val_accuracy: 0.7963
Epoch 4/80
216/216 [==============================] - 312s 1s/step - loss: 1.7397 - accuracy: 0.6713 - val_loss: 1.7826 - val_accuracy: 0.7593
Epoch 5/80
216/216 [==============================] - 311s 1s/step - loss: 4.0512 - accuracy: 0.6759 - val_loss: 1.3894 - val_accuracy: 0.8333
Epoch 6/80
216/216 [==============================] - 311s 1s/step - loss: 1.4982 - accuracy: 0.6574 - val_loss: 1.3571 - val_accuracy: 0.8148
Epoch 7/80
216/216 [==============================] - 311s 1s/step - loss: 1.4637 - accuracy: 

In [ ]:
np.unique(Y)

array([0, 1])

In [ ]:
n0, n1

(array([143, 166, 160, 205,  37,  78,   7,  47, 138, 209, 152, 164, 101,
         32, 212, 249,  79,  46,  91,  76,  82, 100, 121, 189,  31, 188,
         26,  39, 163,  85, 158,  29, 225,  50, 239, 201, 129,  75, 142,
         22, 223, 184, 246,  23, 179, 222,  73,  10, 194, 224, 130, 226,
        165,   2,   5, 141, 144,  15, 192, 206,  27, 173,  65,  14, 247,
        245,  25, 187, 238, 124,  87, 221, 119, 252,  54, 135,  33,  43,
        139,  64,  55, 219, 250, 102,  58, 117,  13, 233, 197, 181,   6,
        191, 104, 213,   0,   4, 108,  12,  56, 241, 235, 159, 162, 199,
        186, 174,  74,  69,  53,  17,  51,  59, 177, 134,  21, 178, 232,
        248, 180,  66, 200, 148, 132,  94, 150, 131,  86, 126,  99, 183,
        103, 156, 218, 171, 215]),
 array([383, 559, 307, 110, 490, 201, 594, 290, 673, 587, 638,  69, 271,
         19, 155, 558,  44, 386, 540, 176, 547, 242,  68, 333, 513, 300,
         83, 163,  35, 337, 238, 245, 181, 371, 526, 252, 517, 387, 626,
        277, 418

In [ ]:
Y[-54:]

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 0])

/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/SEVERITY/CT 0/study_0054.nii.gz
test
0
patient : 0 slice : 0
patient : 0 slice : 1
patient : 0 slice : 2
patient : 0 slice : 3
patient : 0 slice : 4
patient : 0 slice : 5
patient : 0 slice : 6
patient : 0 slice : 7
patient : 0 slice : 8
patient : 0 slice : 9
patient : 0 slice : 10
patient : 0 slice : 11
patient : 0 slice : 12
patient : 0 slice : 13
patient : 0 slice : 14
patient : 0 slice : 15
patient : 0 slice : 16
patient : 0 slice : 17
patient : 0 slice : 18
patient : 0 slice : 19
patient : 0 slice : 20
patient : 0 slice : 21
patient : 0 slice : 22
patient : 0 slice : 23
patient : 0 slice : 24
patient : 0 slice : 25
patient : 0 slice : 26
patient : 0 slice : 27
patient : 0 slice : 28
patient : 0 slice : 29
/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/SEVERITY/CT 0/study_0019.nii.gz
test
0
patient : 1 slice : 0
patient : 1 slice : 1
patient : 1 slice : 2
patient : 1 slice : 3
patient : 1 slice : 4
patient : 1 

In [ ]:
test1

array([273, 662,  49, 473, 251, 532, 152, 530, 444, 679, 164, 298, 303,
       306, 137, 633, 400, 453,   0, 606,  73, 255, 614, 487, 556, 646,
       210])